In [1]:
!pip install transformers[torch] -q
!pip install numpy matplotlib -q

import warnings
warnings.filterwarnings("ignore")

import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import pipeline
import re

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = AutoModelForMaskedLM.from_pretrained('sampathlonka/San-BERT')


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Using cpu device


config.json: 100%|████████████████████████████████████████████████████████████████████████████| 870/870 [00:00<?, ?B/s]
pytorch_model.bin: 100%|████████████████████████████████████████████████████████████| 438M/438M [01:38<00:00, 4.45MB/s]
generation_config.json: 100%|███████████████████████████████████████████████████████████████| 90.0/90.0 [00:00<?, ?B/s]


In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [3]:
from transformers import AutoTokenizer
from transformers import PreTrainedTokenizerFast

In [4]:
tokenizer1=AutoTokenizer.from_pretrained('sampathlonka/San-BERT')
tokenizer1

tokenizer_config.json: 100%|███████████████████████████████████████████████████████████| 394/394 [00:00<00:00, 395kB/s]
vocab.txt: 100%|█████████████████████████████████████████████████████████████████████| 472k/472k [00:00<00:00, 707kB/s]
tokenizer.json: 100%|███████████████████████████████████████████████████████████████| 951k/951k [00:00<00:00, 1.46MB/s]
special_tokens_map.json: 100%|████████████████████████████████████████████████████████████████| 125/125 [00:00<?, ?B/s]


BertTokenizerFast(name_or_path='sampathlonka/San-BERT', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
tokenizer1("अथ तस्य विवाहकौतुकम् ललितम् बिभ्रत एव पार्थिवः|")
#हस्तगामिनीमकरोदिन्दुमतीमिवापराम्

{'input_ids': [2, 2050, 1877, 4120, 29572, 1918, 10867, 1029, 20602, 1812, 12720, 50, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
import os

folder_path = r"D:\github archive\Abstractive\Data\unsupervised\text_files"  # Replace this with the path to your folder

# Get the list of all files in the folder with full path
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [9]:
path=r"D:\github archive\Abstractive\Data\unsupervised\text_files"
from datasets import load_dataset
dataset = load_dataset("text", data_files=files,split='train')

Extracting data files: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.65s/it]
Generating train split: 1854091 examples [00:16, 103718.75 examples/s]

In [13]:
len(dataset)

In [14]:
dataset

In [15]:
dataset['text'][2596]

In [11]:
from tqdm import tqdm

def get_training_corpus():
    for start_idx in tqdm(range(0, len(dataset)), desc="Processing dataset"):
        samples = dataset[start_idx]
        yield samples["text"]

In [12]:
training_corpus = get_training_corpus()

In [ ]:
from tqdm import tqdm

# Assuming get_training_corpus() is defined as mentioned in the previous response

# Wrap your iterator with tqdm
total_samples = len(dataset)
training_corpus_with_progress = tqdm(get_training_corpus(), desc="Tokenizing", total=total_samples)

new_tokenizer = tokenizer1.train_new_from_iterator(training_corpus_with_progress, 52000)

Processing dataset:   2%|▏         | 290462/11832638 [00:29<26:06, 7369.15it/s]

In [15]:
new_tokenizer

NameError: name 'new_tokenizer' is not defined